# RestReq

> class allows us to create the rest request body as well as call. For now `GET, POST, PUT, DELETE` methods are allowed

In [ ]:
#| export
from dataclasses import dataclass
from typing import Union, Optional, Dict
from collections import defaultdict
import requests
import json

In [ ]:
#| export
ENV = defaultdict(lambda: None)
get_env = lambda x : ENV[x] if ENV[x] else ''
def set_env(key, val): ENV[key] = val

In [ ]:
assert get_env('x') == ''
set_env('x', 3)
assert get_env('x') == 3

In [ ]:
#| export
allowed_methods = ["GET","POST","PUT", "DELETE"]

@dataclass
class RestReq:
    method: str
    url: str
    headers: Union[dict,None] = None
    params: Union[dict,None]= None
    body: Union[dict, None] = None
    kwargs: Union[dict, None] = None
    
    @property
    def curl(self):
        
        params_str = "?"+"&".join([f"{key}={value}" for key, value in self.params.items()]) if self.params else ''
            
        curl_cmnd = f"curl -X {self.method.upper()} '{self.url}{params_str}'"

        if self.headers:
            for key, value in self.headers.items():
                curl_cmnd += f" -H '{key}: {value}'"
        
        if self.body :
            curl_cmnd += f" -d '{json.dumps(self.body)}'"
            
        return curl_cmnd

    def __call__(self, **kwargs):
        response = None
        try:
            if self.method.upper() in allowed_methods:
                response = requests.request(self.method, self.url, headers=self.headers, params=self.params, data=json.dumps(self.body), **kwargs)
            else:
                raise "not a valid method"
        
        except Exception as e:
            print(f"Error occurred while making the HTTP request: {e}")
        
        finally:
            return response

    def parse_kwargs(self):
        if self.kwargs:
            for k,v in self.kwargs.items():
                if k == 'cookies' and isinstance(self.kwargs[k], requests.cookies.RequestsCookieJar ):
                    cookies = self.kwargs[k]
                    arr = []
                    for c in cookies:
                        arr.append({
                            'name': c.name,
                            'value': c.value,
                            'domain': c.domain,
                            'path': c.path
                        })
                    
                    self.kwargs[k] = arr

    def __repr__(self):
        """
        print the request details as a JSON object.

        Returns:
            str: A JSON string representation of the request details.
        """
        self.parse_kwargs()
        req_details = {
            'method': self.method,
            'url': self.url,
            'headers': self.headers if self.headers else 'None',
            'params': self.params if self.params else 'None',
            'body': self.body if self.body else 'None',
            'kwargs': self.kwargs if self.kwargs else 'None'
        }
        
        return json.dumps(req_details, indent=4)

In [ ]:
#| export
class RestReqFactory:
    def __init__(self, method, url_provider, headers_provider=lambda:None, params_provider=lambda:None, body_provider=lambda:None):
        self.method = method
        self.url_provider = url_provider
        self.headers_provider = headers_provider
        self.params_provider = params_provider
        self.body_provider = body_provider
        self.kwargs = None
    def __call__(self):
        return RestReq(
            method=self.method,
            url=self.url_provider(),
            headers=self.headers_provider(),
            params=self.params_provider(),
            body=self.body_provider(),
            kwargs=self.kwargs 
        )
        
    def set_req_kwargs(self, kwargs):
        self.kwargs = kwargs

## httpbin testing

In [ ]:
url = lambda: f"{get_env('url')}/post"

head = lambda : {
    "accept": f"{get_env('accept')}",
}

# Create the RestReqFactory instance
req = RestReqFactory(
    method="POST",
    url_provider=url,
    headers_provider=head,
)

In [ ]:
assert req().curl == """curl -X POST '/post' -H 'accept: '"""

Dynammically adding the variable name for url and accept

In [ ]:
ENV['url'] = 'https://httpbin.org'
ENV['accept'] = 'application/json'

In [ ]:
assert req().curl == """curl -X POST 'https://httpbin.org/post' -H 'accept: application/json'"""

In [ ]:
resp = req()()

In [ ]:
assert resp.status_code  == 200

In [ ]:
req()

{
    "method": "POST",
    "url": "https://httpbin.org/post",
    "headers": {
        "accept": "application/json"
    },
    "params": "None",
    "body": "None",
    "kwargs": "None"
}

##  Postman cookies testing

In [ ]:
url = lambda: f"{get_env('postman-url')}/get"

params = lambda : {
    "foo1": f"{get_env('foo1')}",
    "foo2": f"{get_env('foo2')}",
}

# Create the RestReqFactory instance
req = RestReqFactory(
    method="GET",
    url_provider=url,
    params_provider=params,
)

In [ ]:
ENV['foo1'] = 'foo1'
ENV['foo2'] = 'foo2'
ENV['postman-url']  = 'https://postman-echo.com'

In [ ]:
res = req()()

In [ ]:
req.set_req_kwargs({'cookies' : res.cookies})

In [ ]:
req()

{
    "method": "GET",
    "url": "https://postman-echo.com/get",
    "headers": "None",
    "params": {
        "foo1": "foo1",
        "foo2": "foo2"
    },
    "body": "None",
    "kwargs": {
        "cookies": [
            {
                "name": "sails.sid",
                "value": "s%3AUyG_Y_gLShMIWH1w4jqFonJJycCUTljc.ppXEGzO2fQ50ShH3iQXU7mnT2DVMjD1OCsTGrtx1WAo",
                "domain": "postman-echo.com",
                "path": "/"
            }
        ]
    }
}